# Running Bokeh on Jupyter Notebook
This demo shows off an interactive visualization using Bokeh for plotting, and Ipython interactors for widgets. The demo runs entirely inside the Ipython notebook, with no Bokeh server required.
The dropdown offers a choice of trig functions to plot, and the sliders control the frequency, amplitude, and phase.
To run, click on, Cell->Run All in the top menu, then scroll to the bottom and move the sliders.

In [2]:
import numpy as np
from ipywidgets import interact,widgets,interactive
from bokeh.io import push_notebook, output_notebook,show
from bokeh.plotting import figure
import pandas as pd
from bokeh.models.widgets import Slider, Select, TextInput
# Need to change from link from notebook to html once migrated to python script
output_notebook()



Loading BokehJS ...

# OLD SRECNA - ONLY HERE FOR REFERENCE


# THINGS TO DO
## - Need to change the data to new simplified Srecna
## - Need to fix widgets to addd new fucntions for operators
## - Think of a way to compact functions

# New SRECNA used for the model

In [3]:
SRECNA2 =pd.read_excel('SISD.xlsx')
SRECNA3=SRECNA2.dropna(axis=0,how='all')  #Drops rows with all NaN values
Indexes=list(SRECNA3.index.values)
SRECNA3=SRECNA3.drop([Indexes[8],Indexes[20],Indexes[19],Indexes[18],Indexes[17],Indexes[16],Indexes[15]],axis=0)
SRECNA3

,2014,2015,2016,2017
"Student Tuition and Fees, net",6.917035e+05,7.301572e+05,7.810814e+05,8.405870e+05
State educational appropriations,3.185000e+05,3.327860e+05,3.485382e+05,3.676680e+05
Federal Pell grants,3.898800e+04,3.943100e+04,4.012007e+04,4.112307e+04
"Contracts and Grants, net",6.621536e+05,6.983390e+05,6.666769e+05,6.666770e+05
"Educational Activities, and Aux",2.565240e+05,2.697580e+05,2.666261e+05,2.666261e+05
"Private gifts, net",1.841120e+05,2.167340e+05,2.381855e+05,2.524766e+05
Investment income,1.238360e+05,1.299410e+05,1.048660e+05,1.357532e+05
"Other revenues, net",1.134020e+05,1.335001e+05,1.263827e+05,1.152838e+05
Salaries and Wages,1.105063e+06,1.173895e+06,1.199051e+06,1.223032e+06
Benefits,3.896455e+05,4.276760e+05,4.435170e+05,4.597292e+05


In [29]:
newValues=SRECNA3.values[:,:]
#In=list(SRECNA3.index.values)
nextYears=np.zeros((newValues.shape[0],newValues.shape[1]))
Svalues=np.hstack((newValues,nextYears))


## Class for the operating levelers

In [32]:

class Operator:
    """An operator operates on the global set of data  scenario."""

    def __init__(self,data):
        """Create an operator with a given section with the given exit.

        name -- A string; the name of this Place.
        exit -- The Place reached by exiting this Place (may be None).
        """
        self.base_levels=np.array([1.5,2.0,0.0,0.0,2.0,2.0,0.0,1.5,1.5,1.5,0.0,1.5,0.0,0.0])
        self.data=np.copy(data)
        self.years_update=data.shape[1]
    def leveler(self, section, assumption):
        i=4
        while(i < self.years_update):    
            newEntry=self.data[section,i-1]*(1.0+assumption)
            self.data[section,i]=newEntry
            i+=1
        totalRevs=np.sum(self.data[0:8,:],axis=0)
        totalExps=np.sum(self.data[8:,:],axis=0)
        new_data=totalRevs-totalExps
        return new_data
    #Creates case using base_levels for all operators
    def basecase(self):
        section=0
        initial=self.leveler(0,0) # initialized the first 4 values
        for i in self.base_levels:
            initial[4:]=self.leveler(section,i)[4:]
            section+=1
        return initial
            

In [33]:

years=np.linspace(2014,2020,7) # 4 base years and 3 years being operating one. 7 operating years
operator1=Operator(Svalues)
operator2=Operator(Svalues)
y1=operator1.basecase()
y2=operator2.basecase()
model = figure(title="Budget Model", plot_height=600, plot_width=800) 
r = model.line(years,y1, color="#2222aa", line_width=2,line_dash=[4, 4], legend ="Scenario")
base=model.line(years,y2, line_width=2,legend="Base Case")
#Change background color
model.background_fill_color = "white"
# change just some things about the x-axes
model.xaxis.axis_label = "Years"
model.xaxis.axis_line_width = 2
model.xaxis.axis_line_color = "Black"

# change just some things about the y-axes

model.yaxis.axis_label = "Net Income (Thousands)"
model.yaxis.major_label_text_color = "Black"
model.yaxis.major_label_orientation = "horizontal"

# change just some things about the x-grid
model.xgrid.grid_line_color = "Black"
model.xgrid.grid_line_alpha = 0.3
# change just some things about the y-grid
model.ygrid.grid_line_alpha = 0.1
model.ygrid.grid_line_color = "Black"
#Change the outline of the plot box
model.outline_line_width = 10
model.outline_line_alpha = 0.3
model.outline_line_color = "navy"

#Interacting widgets for Revenues
def tuition(assume=0):
    new_data=operator1.leveler(0,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def state(assume=0):
    new_data=operator1.leveler(1,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    

def pell_grants(assume=0):
    new_data=operator1.leveler(2,assume)
    r.data_source.data['y'] = new_data
    push_notebook()

def contracts(assume=0):
    new_data=operator1.leveler(3,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def educational_activities(assume=0):
    new_data=operator1.leveler(4,assume)
    r.data_source.data['y'] = new_data
    push_notebook()

def private_gifts(assume=0):
    new_data=operator1.leveler(5,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def investment_income(assume=0):
    new_data=operator1.leveler(6,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def otherr(assume=0):
    new_data=operator1.leveler(7,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
#Interacting widgets for expenses

def salaries(assume=0):

    new_data=operator1.leveler(8,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def benefits(assume=0):
    new_data=operator1.leveler(9,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def scholarships(assume=0):
    new_data=operator1.leveler(10,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def utilities(assume=0):
    new_data=operator1.leveler(11,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def supplies(assume=0):
    new_data=operator1.leveler(12,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def othere(assume=0):
    new_data=operator1.leveler(13,assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
    
show(model,notebook_handle=True)

#Revenues Levelers
R1=interactive(tuition, assume=widgets.FloatSlider(description="Tuition",min=-1.5, max=4,step=1.5,continuous_update=False))
R2=interactive(state, assume=widgets.FloatSlider(description="State",min=-2.0, max=6.0,step=2.0,continuous_update=False))
R3=interactive(pell_grants, assume=widgets.FloatSlider(description="Pell Grants",min=-5.0, max=5.0,step=2.5,continuous_update=False))
R4=interactive(contracts, assume=widgets.FloatSlider(description="Contracts",min=-5.0, max=5.0,step=2.5,continuous_update=False))
R5=interactive(educational_activities, assume=widgets.FloatSlider(description="Educational Activities",min=-2.0, max=6.0,step=2.0,continuous_update=False))
R6=interactive(private_gifts, assume=widgets.FloatSlider(description="Private Gifts",min=-2.0, max=6.0,step=2.0,continuous_update=False))
R7=interactive(investment_income, assume=widgets.FloatSlider(description="Investment income",min=-10.0, max=10.0,step=5.0,continuous_update=False))
R8=interactive(otherr, assume=widgets.FloatSlider(description="Other",min=-1.5, max=4.0,step=1.5,continuous_update=False))

#Expenses Levelers
E1=interactive(salaries, assume=widgets.FloatSlider(description="Salaries",min=-1.5, max=4.0,step=1.5,continuous_update=False))
E2=interactive(benefits, assume=widgets.FloatSlider(description="Benefits",min=-1.5, max=4.0,step=1.5,continuous_update=False))
E3=interactive(scholarships, assume=widgets.FloatSlider(description="Scholarships and Fellow Ships",min=-5.0, max=5.0,step=2.5,continuous_update=False))
E4=interactive(utilities, assume=widgets.FloatSlider(description="Utilities",min=-1.5, max=4.0,step=1.5,continuous_update=False))
E5=interactive(supplies, assume=widgets.FloatSlider(description="Supplies",min=-5.0, max=5.0,step=2.5,continuous_update=False))
E6=interactive(othere, assume=widgets.FloatSlider(description="Other",min=-5.0, max=5.0,step=2.5,continuous_update=False))

#children = [widgets.Text(description=name) for name in list]
revenues=[R1,R2,R3,R4,R5,R6,R7,R8]
expenses=[E1,E2,E3,E4,E5,E6]
v1=widgets.VBox(revenues)
v2=widgets.VBox(expenses)
children=[v1,v2]
tab = widgets.Tab(children=children)
tab.set_title(0,"Revenues")
tab.set_title(1,"Expenses")




In [31]:
tab
